In [38]:
# The project is telling either the SMS is spam or ham.
# Author: Muhammad Humayun Khan

import pandas as pd
import numpy as np

dataset_path = '../datasets/sms_spam_dataset/spam.csv'

# Load the dataset
df = pd.read_csv(dataset_path, encoding='latin-1')[['v1', 'v2']]

df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [39]:
df.shape

(5572, 2)

In [40]:

# Rename the columns for clarity
df.columns = ['label', 'text']

In [41]:
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [42]:
# Clean labels
df['label'] = df['label'].astype(str).str.strip().str.lower()


In [43]:
# Filter valid rows - only ham and spam
df = df[df['label'].isin(['ham', 'spam'])]

In [44]:
# Reset index
df = df.reset_index(drop=True)


In [45]:
# Map labels
y = df['label'].map({'ham': 0, 'spam': 1})


In [ ]:
# Check for sanity
print(df['label'].value_counts())
print("Any NaNs in y:", y.isnull().sum())

# the following data showing the more than 80cent of the dataset as ham and it will effect the training towards ham


label
ham     4825
spam     747
Name: count, dtype: int64
Any NaNs in y: 0


In [46]:
# finding the missing rows
df.isnull().sum()

label    0
text     0
dtype: int64

In [47]:
# finding the duplicate rows
df.duplicated().sum()

403

In [ ]:
# Now after exploring the dataset, the next steps are
# 1. Text preprocessing which includes: lowercasing, removing punctuation, removing stopwords, tokenization, stemming/lemmatization
# 2. Text Representation/Feature extraction: to convert text to numerical form for ML Model: using TF-IDF or Count Vectorization
# 3. Model Training: Encode labels spam = 1, ham = 0, split data into train/test and train the model using Random Forest
# 4. Evaluate the model using accuracy, precision, recall, f1-score

In [49]:
# Now we need to clean the text which is Text Preprocessing
# Create the function to apply all the text preprocessing steps
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    # Lowercase
    text = text.lower()

    # Remove punctuation, numbers, special characters
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove full stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [50]:
# apply on the dataframe
df['text'] = df['text'].apply(preprocess_text)

In [51]:
# check the output
df.head()

,label,text
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry wkly comp win fa cup final tkts st ...
3,ham,u dun say early hor u c already say
4,ham,nah dont think go usf life around though


In [52]:
df.sample(10)

,label,text
1321,ham,wake long ago already dunno thing
2556,ham,fuck babe happened come never came back
4639,ham,hide anythiing keeping distance
1680,ham,okay wait ah
1951,ham,haha really oh deduct lesson tmr
1269,ham,tee hee lecture cheery bye bye
5554,ham,well keep mind ive got enough gas one round tr...
1650,ham,dont file bagi work called mei tell find anyth...
702,ham,important prevent dehydration giving enough fluid
1422,spam,congratulation ur awarded either cd gift vouch...


In [53]:
# Now the next step after the text preprocessing is to convert the text to numerical form for ML Model
from sklearn.feature_extraction.text import TfidfVectorizer


# Initialize vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the cleaned text
X = vectorizer.fit_transform(df['text'])

print("TF-IDF shape:", X.shape) # the output will show the number of rows and unique words such as rows = 5572 and columns = 7855



TF-IDF shape: (5572, 7855)


In [55]:
# Now train/test split the data and train the model
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [67]:
# Now train the model using Random Forest
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

# Initialize the model
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)

# Train the model
rf_model.fit(X_train, y_train)



,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [68]:
# predict the model on test
rf_preds = rf_model.predict(X_test)

In [69]:
# Evaluate the model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Random Forest Evaluation:")
print("Accuracy:", accuracy_score(y_test, rf_preds))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_preds))
print("Classification Report:\n", classification_report(y_test, rf_preds))


Random Forest Evaluation:
Accuracy: 0.9730941704035875
Confusion Matrix:
 [[965   0]
 [ 30 120]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       965
           1       1.00      0.80      0.89       150

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [ ]:
# Now lets predict a new sample message or spam or ham for the Random Forest Classifier
def predict_message(text):
    
    def preprocess_text(text):
        text = text.lower()
        text = ''.join(char for char in text if char.isalnum() or char.isspace())
        tokens = text.split()
        stopwords = set(['the', 'is', 'in', 'and', 'to', 'a', 'of', 'that', 'it', 'this', 'for', 'with'])
        tokens = [word for word in tokens if word not in stopwords]
        return ' '.join(tokens)

    cleaned_text = preprocess_text(text)

    # TF-IDF transform
    text_vector = vectorizer.transform([cleaned_text])  

    # Predict using trained model
    prediction = rf_model.predict(text_vector)

    return "Spam" if prediction[0] == 1 else "Ham"

print(predict_message("Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/123 to claim now!"))
print(predict_message("Win a brand new car by texting WIN to 90909"))
print(predict_message("Hey, are we still meeting at 6pm today?"))



Spam
Ham
Ham
Ham
Ham
Ham
Ham
